Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf


#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'NFLX'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, '2000-11-08', '2021-11-08' )
#TODO
df = df.iloc[:, :4]
#TODO
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2002-05-23,1.156429,1.242857,1.145714,1.196429
2002-05-24,1.214286,1.225000,1.197143,1.210000
2002-05-28,1.213571,1.232143,1.157143,1.157143
2002-05-29,1.164286,1.164286,1.085714,1.103571
2002-05-30,1.107857,1.107857,1.071429,1.071429
...,...,...,...,...
2021-11-01,689.059998,689.969971,676.539978,681.169983
2021-11-02,683.109985,687.679993,673.820007,677.719971
2021-11-03,677.270020,689.390015,677.270020,688.289978


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_30_Rolling'] = df['Close'].rolling(window=30).mean()
df['Close_45_Rolling'] = df['Close'].rolling(window=45).mean()
df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1'] = df['Open']-df['Open'].shift(1)
df['Range']= df['High']-df['Low']
df = df.dropna()
X = df
X



#df['Close_15_Rolling'] = df['Close'].rolling(window=30).mean()
#TODO
#df['Open_1_Change'] = df['Open']-df['Open'].shift(1)

#TODO
#df = df.dropna()
#TODO
#X = df
#TODO
#X

,Open,High,Low,Close,Close_30_Rolling,Close_45_Rolling,Close_15_Rolling,Open_1,Range
Date,,,,,,,,,
2002-07-26,1.050000,1.058571,0.850000,0.871429,1.075238,1.094492,1.132286,-0.075000,0.208571
2002-07-29,0.896429,0.924286,0.850000,0.876429,1.071571,1.087381,1.112333,-0.153571,0.074286
2002-07-30,0.893571,0.942857,0.893571,0.914286,1.071310,1.080810,1.095905,-0.002858,0.049286
2002-07-31,0.928571,0.960000,0.905714,0.953571,1.072738,1.076286,1.081905,0.035000,0.054286
2002-08-01,0.950000,0.960000,0.877857,0.891429,1.070738,1.071571,1.056714,0.021429,0.082143
...,...,...,...,...,...,...,...,...,...
2021-11-01,689.059998,689.969971,676.539978,681.169983,630.091996,615.766663,652.364661,16.000000,13.429993
2021-11-02,683.109985,687.679993,673.820007,677.719971,633.577995,618.245329,655.883325,-5.950012,13.859985
2021-11-03,677.270020,689.390015,677.270020,688.289978,636.832660,620.891995,659.785323,-5.839966,12.119995


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] < 0, -1, 1)
y
#TODO

array([ 1,  1,  1, ..., -1, -1,  1])

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size= index)

#index = int(0.7*len(X))
#X_train, X_test, y_train, y_test = X[:index], X[index:], y[:index], y[index:]
#TODO

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)
#TODO

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.0032319340595289335]
1,High,[0.009126232479232986]
2,Low,[0.02836962813622846]
3,Close,[-0.05190161631014219]
4,Close_30_Rolling,[-0.025328029380380528]
5,Close_45_Rolling,[0.02385972347368382]
6,Close_15_Rolling,[0.014296164645546509]
7,Open_1,[0.004631212643728352]
8,Range,[-0.019243395656941026]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test) #TODO
predictions = logistic.predict(X_test)
#TODO

In [10]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.
metrics.confusion_matrix(y_test, predictions)
#TODO

array([[ 265, 1407],
       [ 218, 1509]], dtype=int64)

In [11]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
logistic.score(X_test, y_test)
#TODO

0.5219182112385996

In [12]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X,y,cv=5)
#cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=5)
#cross_val = cross_val_score(logistic,X_test,y_test,cv=5)
#cross_val = cross_val_score(logistic, X, y, scoring='accuracy', cv = 5)
cross_val
#TODO

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

array([0.51337449, 0.50978373, 0.51390319, 0.54067971, 0.55097837])